In [1]:
import numpy as np
import pulp as plp
import pandas as pd 
from pulp import *
from timeit import default_timer as timer
df = pd.read_csv (r"C:\Users\Sophie\Downloads\players.csv")
df['Salary'] = df['Salary']/1000000

Using license file C:\Users\Sophie\gurobi.lic
Academic license - for non-commercial use only - expires 2021-01-01
No parameters matching '_test' found


In [2]:
##### Expected Salary
E_S = []

players = range(len(df))
I = players

for i in I:
    PER = df.iloc[i, 11]
    age = df.iloc[i, 6]
    WS = df.iloc[i, 25]
    eFG = df.iloc[i, 40]
    
    Estimated_salary = -0.0394 + 0.2787*PER + 0.0269*age + 1.1325*WS + -0.0229*eFG
    E_S.append(Estimated_salary) #estimated salary column is added

df2 = df.assign(ES = E_S)
df2 #new dataframe


##### Expected win shares
E_W = []

for i in I:
    WS = df.iloc[i, 26] #WS/48 variable in dataset
    MP = df.iloc[i, 10] #MP variable in dataset
    
    expected_wins = (MP/2)*(WS/48)*MP*(1 - (1- 1/MP)*(1- 1/MP)) #EW formula from Gurobi
    E_W.append(expected_wins)

df3 = df2.assign(EW = E_W)


###### player position matrix

BB2 = df3[['ID', 'Player Name', 'Pos', 'Salary', 'PER', 'ES', 'WS', 'G', 'EW', 'GS', 'MP']]

players_POS = BB2.assign(PG = [1 if a == 'PG' else 0 for a in BB2['Pos']])
players_POS = players_POS.assign(SG = [1 if a == 'SG' else 0 for a in players_POS['Pos']])
players_POS = players_POS.assign(SF = [1 if a == 'SF' else 0 for a in players_POS['Pos']])
players_POS = players_POS.assign(PF = [1 if a == 'PF' else 0 for a in players_POS['Pos']])
players_POS = players_POS.assign(C = [1 if a == 'C' else 0 for a in players_POS['Pos']])


#### Starting players
players_POS['GS_GP'] = players_POS.GS / players_POS.G
top_starting = players_POS['GS_GP'].quantile(0.8) ## this means players that start 100% of their games

players_POS = players_POS.assign(Start_P = [1 if a >= top_starting else 0 for a in players_POS['GS_GP']] )
 

In [3]:
########################### DECISION VARIBLES ################################

#Establishing the variables 
positions = ['PG', 'SG', 'SF', 'PF', 'C']
players = range(len(players_POS))

I = players
J = range(len(positions))


X = plp.LpVariable.dicts(name = 'x', indexs=(I), cat=plp.LpBinary)


############################# CONSTRAINTS ##################################
luxury_taxes = []
optimal_solution = []
l = [i for i in range(200,-100, -10)]

start_time = timer()
for i in l:
    
    #Setting up the problem
    prob1 = LpProblem(name = "basketball_positions")
    prob1.sense = LpMaximize
    
    #1 - max 15 players
    prob1 += lpSum(X[i] for i in I) <= 15

    #2 - salary cap 
    SALARY_CAP = 99.09
    luxury_money = i
    hard_cap = (luxury_money/1.5) + SALARY_CAP
    salary = players_POS['Salary'].tolist()

    prob1 += lpSum(X[j]*salary[j] for j in I) <= hard_cap


    #3 - only select players if they are undervalued
    estimated_salary = players_POS['ES'].tolist()
    prob1 += lpSum(X[j]*salary[j] - X[j]*estimated_salary[j] for j in J) <= 0

    #4 - position constraints
    pointG = players_POS['PG'].tolist()
    prob1 += lpSum(X[i]*pointG[i] for i in I) == 1
    shootingG = players_POS['SG'].tolist()
    prob1 += lpSum(X[i]*shootingG[i] for i in I) == 2
    shootingF = players_POS['SF'].tolist()
    prob1 += lpSum(X[i]*shootingF[i] for i in I) == 3
    pointF = players_POS['PF'].tolist()
    prob1 += lpSum(X[i]*pointF[i] for i in I) == 4
    center = players_POS['C'].tolist()
    prob1 += lpSum(X[i]*center[i] for i in I) == 5

    #5 - players must have been in at least the 10th percentile for minutes played 
    threshold = players_POS['MP'].quantile(0.1) #287 minutes played
    minutes_played = players_POS['MP'].tolist()
    prob1 += lpSum(X[i]*minutes_played[i] - threshold >= 0 for i in I) #they must have played more minutes then the threshold


    #6 - Games started constraint -- NOT SURE IF WE SHOULD REMOVE THIS OR NOT? 
    total_games_started = 390
    games_started = players_POS['GS'].tolist()
    prob1 += lpSum(X[i]*games_started[i] for i in I) <= 390


    #6 - Starting players constraint
    starting_player = players_POS['Start_P'].tolist()
    prob1 += lpSum(X[i]*pointG[i]*starting_player[i] for i in I) == 1
    prob1 += lpSum(X[i]*shootingG[i]*starting_player[i] for i in I) == 1
    prob1 += lpSum(X[i]*shootingF[i]*starting_player[i] for i in I) == 1
    prob1 += lpSum(X[i]*pointF[i]*starting_player[i] for i in I) == 1
    prob1 += lpSum(X[i]*center[i]*starting_player[i] for i in I) == 1



    ############################### OBJECTIVE ###################################

    #maximizing the PER and Win shares
    PER = players_POS['PER'].tolist()
    EW = players_POS['EW'].tolist()
    prob1 += lpSum(X[i]*(PER[i]+EW[i]) for i in I)


    ##############################
    prob1.solve(GUROBI(timeLimit=1))
    luxury_taxes.append(i)
    optimal_solution.append(value(prob1.objective))
    
end_time = timer()


Changed value of parameter TimeLimit to 1.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 14 rows, 394 columns and 1598 nonzeros
Model fingerprint: 0xc78ba04c
Variable types: 0 continuous, 394 integer (0 binary)
Coefficient statistics:
  Matrix range     [2e-02, 8e+01]
  Objective range  [5e+00, 4e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+02]
Found heuristic solution: objective 239.7385312
Presolve removed 2 rows and 168 columns
Presolve time: 0.02s
Presolved: 12 rows, 226 columns, 637 nonzeros
Found heuristic solution: objective 413.7987917
Variable types: 0 continuous, 226 integer (226 binary)

Root relaxation: objective 4.424175e+02, 30 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node 

C:\Users\Sophie\anaconda3\lib\site-packages\pulp\pulp.py:1521: UserWarning: Overwriting previously set objective.
  warnings.warn("Overwriting previously set objective.")


Gurobi status= 2
Changed value of parameter TimeLimit to 1.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 14 rows, 394 columns and 1598 nonzeros
Model fingerprint: 0x6f8121a9
Variable types: 0 continuous, 394 integer (0 binary)
Coefficient statistics:
  Matrix range     [2e-02, 8e+01]
  Objective range  [5e+00, 4e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+02]
Found heuristic solution: objective 239.7385312
Presolve removed 2 rows and 168 columns
Presolve time: 0.00s
Presolved: 12 rows, 226 columns, 637 nonzeros
Found heuristic solution: objective 413.7987917
Variable types: 0 continuous, 226 integer (226 binary)

Root relaxation: objective 4.422669e+02, 23 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd


Root relaxation: objective 4.389219e+02, 28 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  438.92188    0    4  413.79879  438.92188  6.07%     -    0s
H    0     0                     429.3741771  438.92188  2.22%     -    0s
H    0     0                     435.5010000  438.92188  0.79%     -    0s
H    0     0                     437.8613542  438.92188  0.24%     -    0s

Cutting planes:
  GUB cover: 1

Explored 1 nodes (28 simplex iterations) in 0.03 seconds
Thread count was 8 (of 8 available processors)

Solution count 5: 437.861 435.501 429.374 ... 239.739

Optimal solution found (tolerance 1.00e-04)
Best objective 4.378613541667e+02, best bound 4.378613541667e+02, gap 0.0000%
Gurobi status= 2
Changed value of parameter TimeLimit to 1.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)


  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+02]
Found heuristic solution: objective 239.7385312
Presolve removed 2 rows and 168 columns
Presolve time: 0.00s
Presolved: 12 rows, 226 columns, 637 nonzeros
Found heuristic solution: objective 412.3754063
Variable types: 0 continuous, 226 integer (226 binary)

Root relaxation: objective 4.322535e+02, 31 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  432.25347    0    4  412.37541  432.25347  4.82%     -    0s
H    0     0                     430.7922604  432.25347  0.34%     -    0s

Cutting planes:
  Cover: 1

Explored 1 nodes (31 simplex iterations) in 0.03 seconds
Thread count was 8 (of 8 available processors)

Solution count 3: 430.792 412.375 239.739 

Optimal solution found (tolerance 1.00e-04)
Best objective 4.307922604167e+02, best bound 4.307922604167e+02, gap 0.0000%
Gu


     0     0  423.65149    0    4  405.10746  423.65149  4.58%     -    0s
H    0     0                     412.2431042  423.65149  2.77%     -    0s
H    0     0                     417.1471667  423.65149  1.56%     -    0s
H    0     0                     419.9618229  423.65149  0.88%     -    0s
H    0     0                     419.9953542  423.65149  0.87%     -    0s
     0     0  422.26050    0   10  419.99535  422.26050  0.54%     -    0s
     0     0  422.26050    0    2  419.99535  422.26050  0.54%     -    0s
     0     0  422.26050    0    8  419.99535  422.26050  0.54%     -    0s
H    0     0                     420.6420625  422.26050  0.38%     -    0s
     0     0  422.26050    0    8  420.64206  422.26050  0.38%     -    0s
     0     0  422.26050    0    9  420.64206  422.26050  0.38%     -    0s
     0     0  421.25372    0    9  420.64206  421.25372  0.15%     -    0s
     0     0  420.64206    0    5  420.64206  420.64206  0.00%     -    0s

Cutting planes:
  Gomor

 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  413.90612    0    3  364.80647  413.90612  13.5%     -    0s
H    0     0                     403.3709688  413.90612  2.61%     -    0s
H    0     0                     405.3113542  413.90612  2.12%     -    0s
H    0     0                     406.7820104  413.90612  1.75%     -    0s
H    0     0                     411.3646250  413.90612  0.62%     -    0s

Cutting planes:
  Cover: 2

Explored 1 nodes (26 simplex iterations) in 0.03 seconds
Thread count was 8 (of 8 available processors)

Solution count 6: 411.365 406.782 405.311 ... 236.514

Optimal solution found (tolerance 1.00e-04)
Best objective 4.113646250000e+02, best bound 4.113646250000e+02, gap 0.0000%
Gurobi status= 2
Changed value of parameter TimeLimit to 1.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threa

Presolve time: 0.00s
Presolved: 12 rows, 226 columns, 637 nonzeros
Found heuristic solution: objective 351.5880417
Variable types: 0 continuous, 226 integer (226 binary)

Root relaxation: objective 4.000372e+02, 25 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  400.03717    0    4  351.58804  400.03717  13.8%     -    0s
H    0     0                     387.1799271  400.03717  3.32%     -    0s
H    0     0                     387.3090417  400.03717  3.29%     -    0s
H    0     0                     390.9562604  400.03717  2.32%     -    0s
H    0     0                     391.1944479  400.03717  2.26%     -    0s
H    0     0                     393.3716354  400.03717  1.69%     -    0s
     0     0  394.22847    0   11  393.37164  394.22847  0.22%     -    0s
     0     0  394.22847    0    3  393.37164  394.22847  0.22%     -    0s
     0  

  Matrix range     [2e-02, 8e+01]
  Objective range  [5e+00, 4e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+02]
Presolve removed 2 rows and 176 columns
Presolve time: 0.00s
Presolved: 12 rows, 218 columns, 618 nonzeros
Variable types: 0 continuous, 218 integer (218 binary)
Found heuristic solution: objective 328.4368125

Root relaxation: objective 3.661176e+02, 23 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  366.11761    0    4  328.43681  366.11761  11.5%     -    0s
H    0     0                     362.3903854  366.11761  1.03%     -    0s
H    0     0                     363.3620104  366.11761  0.76%     -    0s

Cutting planes:
  Cover: 1
  RLT: 1

Explored 1 nodes (23 simplex iterations) in 0.03 seconds
Thread count was 8 (of 8 available processors)

Solution count 3: 363.362 362.39 328.437 

Optimal solution foun

In [4]:
Time_it_Took = round(end_time - start_time,5)
print("Time:{}s". format(Time_it_Took))

Time:3.38699s


In [8]:
import plotly.express as px

df = pd.DataFrame()
df['taxes'] = luxury_taxes
df['PER_WS'] = optimal_solution


#plot of optimal solution as a function of luxury tax allowance
px.scatter(df, x = 'taxes', y = 'PER_WS', trendline = 'lowess',
          labels={
              'taxes' : 'luxury tax allowance (millions)',
              'PER_WS' : 'optimal solution value'
          },
                     title = 'Luxury tax constraint in pulp model')